# **Advanced Camera Calibration With OpenCV Reza Barzegar**

* **Reza Barzegar**
* **Contacting Me**:
  * Telegram ID: [@Lincraf_t](https://t.me/@Lincraf_t)

# 📸 Advanced Camera Calibration Suite: A Deep Dive with OpenCV

Welcome to this presentation. Today, we will explore a complete computer vision tool designed for **camera calibration**. This script uses OpenCV and Python to analyze images of a chessboard pattern, calculate a camera's intrinsic properties, and correct for lens distortion.

**Project at a Glance:**

* **Technology Stack**:
    * **Core Libraries**: OpenCV and NumPy.
    * **Functionality**: A command-line tool that provides an end-to-end pipeline for camera calibration and analysis.

* **Key Features We Will Explore**:
    1.  **Live Calibration**: Capturing images directly from a webcam to generate calibration data.
    2.  **Interactive Undistortion**: A real-time view with trackbar controls to visualize and adjust the lens correction.
    3.  **Accuracy Visualization**: A live reprojection error overlay to interactively assess the quality of the calibration.
    4.  **Data Export**: The ability to save calibration parameters in both machine-readable (`.npz`) and human-readable (`.json`) formats.

This notebook will guide you through the entire script, explaining how each component works, before showing you how to run it.

---

### 🗺️ Our Presentation Roadmap

Our goal is to deconstruct this computer vision tool to understand it fully. We will follow this agenda:

1.  **Setup**: Install the necessary Python libraries for computer vision.
2.  **Code Reconstruction**: Write the complete, professionally structured Python script into our Colab environment.
3.  **In-Depth Analysis**: Break down the `CameraCalibrator` class, explaining the core computer vision concepts behind each of its functions.
4.  **Usage and Demonstration**: Explain how to run the tool from the command line in its various modes and what to expect from each.

Let's begin by setting up our environment.

---

## Part 2: ⚙️ Environment Setup

This computer vision tool has very few dependencies, relying on the core libraries of the Python scientific computing ecosystem.

* **`opencv-python`**: The industry-standard library for computer vision tasks. We will use it for capturing video, detecting patterns, and performing all calibration and image manipulation calculations.
* **`numpy`**: The fundamental package for numerical computing in Python. OpenCV uses NumPy arrays to store and manipulate image data and matrices.

Let's install them.

---

In [ ]:
!pip install opencv-python numpy

## Part 3: 🏗️ Code Reconstruction

Now, we will create the single, complete Python script for our application. The code has been refactored into a robust `CameraCalibrator` class. This Object-Oriented approach encapsulates all the functionality, making the code cleaner, more reusable, and easier to maintain.

The cell below contains the entire application.

---

In [ ]:
%%writefile camera_calibrator.py

import cv2
import numpy as np
import os
import argparse
import glob
import json

def draw_distortion_grid(frame, cam_matrix, dist_coeffs, step=100, color=(0, 0, 255)):
    """
    Draws a visual grid over the frame.
    The grid is distorted according to the distortion coefficients.
    """
    h, w = frame.shape[:2]

    # Generate grid points
    for x in range(0, w, step):
        pts = np.array([[x, y] for y in range(0, h, step)], dtype=np.float32)
        pts = pts.reshape(-1, 1, 2)
        # Undistort points
        pts_undistorted = cv2.undistortPoints(pts, cam_matrix, dist_coeffs, P=cam_matrix)
        pts_undistorted = pts_undistorted.reshape(-1, 2)
        for i in range(len(pts_undistorted)-1):
            cv2.line(frame, tuple(pts_undistorted[i].astype(int)), tuple(pts_undistorted[i+1].astype(int)), color, 1)

    for y in range(0, h, step):
        pts = np.array([[x, y] for x in range(0, w, step)], dtype=np.float32)
        pts = pts.reshape(-1, 1, 2)
        pts_undistorted = cv2.undistortPoints(pts, cam_matrix, dist_coeffs, P=cam_matrix)
        pts_undistorted = pts_undistorted.reshape(-1, 2)
        for i in range(len(pts_undistorted)-1):
            cv2.line(frame, tuple(pts_undistorted[i].astype(int)), tuple(pts_undistorted[i+1].astype(int)), color, 1)

    return frame


class CameraCalibrator:
    """
    A class to handle camera calibration and real-time undistortion.

    This tool can:
    1. Calibrate a camera using a live video feed (`--mode live`).
    2. Calibrate from a directory of pre-captured images (`--mode dir`).
    3. Display a real-time, side-by-side view of the original and undistorted
       camera feed using pre-existing calibration data (`--mode undistort`).
    """

    def __init__(self, rows=7, cols=7, square_size=25.0, image_dir="calib_images", output_file="calibration_data.npz"):
        """
        Initializes the CameraCalibrator.

        Args:
            rows (int): The number of inner corners on the chessboard's vertical axis.
            cols (int): The number of inner corners on the chessboard's horizontal axis.
            square_size (float): The size of one square on the chessboard in millimeters.
            image_dir (str): The directory to save/load calibration images.
            output_file (str): The file path to save/load the calibration data.
        """
        self.chessboard_dims = (cols, rows)
        self.square_size = square_size
        self.image_dir = image_dir
        self.output_file = output_file

        # Termination criteria for corner refinement
        self.criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

        # Prepare 3D object points
        self.objp = np.zeros((self.chessboard_dims[0] * self.chessboard_dims[1], 3), np.float32)
        self.objp[:, :2] = np.mgrid[0:self.chessboard_dims[0], 0:self.chessboard_dims[1]].T.reshape(-1, 2)
        self.objp *= self.square_size

        self.obj_points = []  # 3D points in real world space
        self.img_points = []  # 2D points in image plane.

        self._ensure_dir_exists(self.image_dir)

    def _ensure_dir_exists(self, path):
        """Checks if a directory exists, and creates it if it doesn't."""
        if not os.path.isdir(path):
            os.makedirs(path)
            print(f"Directory '{path}' was created.")

    @staticmethod
    def is_frame_sharp(frame, threshold=100):
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        lap = cv2.Laplacian(gray, cv2.CV_64F)
        variance = lap.var()
        return variance > threshold

    def calibrate_from_live_feed(self, camera_index=0):
        """
        Captures images from a live camera feed and then runs calibration.

        Press 's' to save a frame when the chessboard is detected.
        Press 'q' to quit capture and proceed to calibration.
        """
        print("\nStarting live calibration capture...")
        print("Press 's' to save a frame when a chessboard is detected.")
        print("Press 'q' to finish capturing and start calibration.")

        cap = cv2.VideoCapture(camera_index)
        if not cap.isOpened():
            print(f"Error: Could not open camera with index {camera_index}.")
            return

        saved_image_count = 0
        while True:
            ret, frame = cap.read()
            if not ret:
                print("Error: Failed to grab frame from camera.")
                break

            copy_frame = frame.copy()
            # In calibrate_from_live_feed
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # Find the chessboard corners with robustness flags
            found, corners = cv2.findChessboardCorners(
                gray,
                self.chessboard_dims,
                cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_NORMALIZE_IMAGE
            )

            # If found, refine and draw corners
            if found:
                refined_corners = cv2.cornerSubPix(
                    gray, corners, (11, 11), (-1, -1), self.criteria
                )
                cv2.drawChessboardCorners(frame, self.chessboard_dims, refined_corners, found)
            else:
                cv2.putText(
                    frame, "No chessboard detected", (30, 70),
                    cv2.FONT_HERSHEY_PLAIN, 1.4, (0, 0, 255), 2
                )

            cv2.putText(
                frame,
                f"Saved Images: {saved_image_count}",
                (30, 40), cv2.FONT_HERSHEY_PLAIN, 1.4, (0, 255, 0), 2, cv2.LINE_AA
            )
            cv2.imshow("Camera Feed - Press 's' to save, 'q' to quit", frame)

            key = cv2.waitKey(1) & 0xFF

            if key == ord('q'):
                break
            elif key == ord('s') and found:
                if not CameraCalibrator.is_frame_sharp(copy_frame, threshold=100):
                    print("Frame too blurry. Skipping...")
                    continue  # skip saving this frame

                img_path = os.path.join(self.image_dir, f"image_{saved_image_count}.png")
                cv2.imwrite(img_path, copy_frame)
                print(f"Saved: {img_path}")
                saved_image_count += 1

        cap.release()
        cv2.destroyAllWindows()

        if saved_image_count > 0:
            print(f"\nCaptured {saved_image_count} images. Now calibrating from directory '{self.image_dir}'...")
            self.calibrate_from_directory()
        else:
            print("\nNo images were saved. Calibration aborted.")

    def calibrate_from_directory(self):
        """
        Calibrates the camera using images from the specified directory.
        """
        images = glob.glob(os.path.join(self.image_dir, '*.png'))
        if not images:
            images = glob.glob(os.path.join(self.image_dir, '*.jpg'))

        if not images:
            print(f"Error: No '.png' or '.jpg' images found in the directory '{self.image_dir}'.")
            return

        print(f"Found {len(images)} images. Processing...")

        gray_shape = None
        for fname in images:
            img = cv2.imread(fname)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            gray_shape = gray.shape[::-1]

            # Find the chessboard corners
            ret, corners = cv2.findChessboardCorners(gray, self.chessboard_dims, None)

            if ret:
                self.obj_points.append(self.objp)
                refined_corners = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), self.criteria)
                self.img_points.append(refined_corners)
            else:
                print(f"Chessboard not found in {fname}. Skipping.")

        cv2.destroyAllWindows()

        if not self.obj_points:
            print("Calibration failed. Could not detect the chessboard in any of the images.")
            return

        print("\nPerforming camera calibration...")
        try:
            ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(
                self.obj_points, self.img_points, gray_shape, None, None
            )

            if not ret:
                print("Calibration was unsuccessful.")
                return

            print("Calibration successful!")
            print("\nCamera Matrix:\n", mtx)
            print("\nDistortion Coefficients:\n", dist)

            self.save_parameters(mtx, dist, rvecs, tvecs)
        except cv2.error as e:
            print(f"An OpenCV error occurred during calibration: {e}")
            print(
                "This can happen if the chessboard pattern is not consistently visible or the images are of poor quality.")

    def undistort_live_feed(self, camera_index=0):
        """
        Displays a live, side-by-side comparison of the original and
        undistorted camera feed using saved calibration data, with grid overlay
        and performance optimization using remap.
        """
        print(f"Loading calibration data from '{self.output_file}'...")
        if not os.path.exists(self.output_file):
            print(f"Error: Calibration file not found at '{self.output_file}'")
            print("Please run the calibration first using '--mode live' or '--mode dir'.")
            return

        data = np.load(self.output_file)
        cam_matrix = data["camMatrix"]
        dist_coeffs = data["distCoef"]

        print("Calibration data loaded. Starting live undistortion feed...")
        print("Press 'q' to quit.")

        cap = cv2.VideoCapture(camera_index)
        if not cap.isOpened():
            print(f"Error: Could not open camera with index {camera_index}.")
            return

        # Initialize undistort maps (once)
        ret, frame = cap.read()
        if not ret:
            print("Error: Failed to grab initial frame.")
            cap.release()
            return

        h, w = frame.shape[:2]
        map1, map2 = cv2.initUndistortRectifyMap(cam_matrix, dist_coeffs, None, cam_matrix, (w, h), cv2.CV_32FC1)

        while True:
            ret, frame = cap.read()
            if not ret:
                print("Error: Failed to grab frame from camera.")
                break

            # --- Performance optimized undistortion ---
            undistorted_frame = cv2.remap(frame, map1, map2, interpolation=cv2.INTER_LINEAR)

            # --- Overlay grids ---
            # Red grid on original frame
            frame_with_grid = frame.copy()
            step = 100
            for x in range(0, frame.shape[1], step):
                cv2.line(frame_with_grid, (x, 0), (x, frame.shape[0] - 1), (0, 0, 255), 1)
            for y in range(0, frame.shape[0], step):
                cv2.line(frame_with_grid, (0, y), (frame.shape[1] - 1, y), (0, 0, 255), 1)

            # Green grid on undistorted frame
            undistorted_with_grid = draw_distortion_grid(undistorted_frame.copy(), cam_matrix, dist_coeffs, step=100,
                                                         color=(0, 255, 0))

            # --- Side-by-side display ---
            h1, w1 = frame.shape[:2]
            h2, w2 = undistorted_frame.shape[:2]
            combined_view = np.zeros((max(h1, h2), w1 + w2, 3), dtype=np.uint8)
            combined_view[:h1, :w1, :] = frame_with_grid
            combined_view[:h2, w1:w1 + w2, :] = undistorted_with_grid

            # Add labels
            cv2.putText(combined_view, "Original (Distorted)", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            cv2.putText(combined_view, "Corrected (Undistorted)", (w1 + 10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

            cv2.imshow("Live Undistortion", combined_view)

            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()

    def save_parameters(self, camera_matrix, dist_coeffs, rvecs, tvecs):
        """Saves the calibration parameters to a .npz file."""
        print(f"\nSaving calibration data to '{self.output_file}'...")
        np.savez(
            self.output_file,
            camMatrix=camera_matrix,
            distCoef=dist_coeffs,
            rVector=rvecs,
            tVector=tvecs
        )
        print("Data saved successfully.")


def main():
    """Main function to parse arguments and run the calibrator."""
    parser = argparse.ArgumentParser(description="Camera Calibration and Undistortion Tool")
    parser.add_argument(
        '--mode', type=str, required=True, choices=['live', 'dir', 'undistort'],
        help="'live' for capture, 'dir' for batch calibration, 'undistort' for live correction."
    )
    parser.add_argument(
        '--rows', type=int, default=7, help="Number of inner corners on the chessboard's vertical axis."
    )
    parser.add_argument(
        '--cols', type=int, default=7, help="Number of inner corners on the chessboard's horizontal axis."
    )
    parser.add_argument(
        '--size', type=float, default=25.0, help="The size of a chessboard square in mm."
    )
    parser.add_argument(
        '--path', type=str, default="calib_images", help="Directory to save/load calibration images."
    )
    parser.add_argument(
        '--output', type=str, default="calibration_data.npz", help="Output file for calibration data."
    )
    args = parser.parse_args()

    calibrator = CameraCalibrator(
        rows=args.rows,
        cols=args.cols,
        square_size=args.size,
        image_dir=args.path,
        output_file=args.output
    )

    if args.mode == 'live':
        calibrator.calibrate_from_live_feed()
    elif args.mode == 'dir':
        calibrator.calibrate_from_directory()
    elif args.mode == 'undistort':
        calibrator.undistort_live_feed()


if __name__ == '__main__':
    main()

## Part 4: 🧠 In-Depth Analysis of the Application

With the script in place, let's break down the key computer vision concepts and code sections.

### 4.1 The `CameraCalibrator` Class

The entire logic is encapsulated within this class. The `__init__` method sets up the essential parameters:

* **`chessboard_dims`**: The number of *inner* corners of the chessboard (e.g., a 9x6 board has 8x5 inner corners).
* **`objp`**: The "object points". This is a crucial concept. We create an ideal, 3D representation of the chessboard corners in a perfect grid, assuming it lies on the Z=0 plane. The goal of calibration is to find the mapping between these perfect 3D points and the distorted 2D points detected in the image.

### 4.2 Calibration Logic (`calibrate_from_directory`)

This is the core of the calibration process. It iterates through images and:
1.  **Finds Corners**: Uses `cv2.findChessboardCorners()` to locate the corners of the chessboard pattern in the 2D image.
2.  **Refines Corners**: Uses `cv2.cornerSubPix()` to find the corner locations with sub-pixel accuracy, which is critical for a good calibration.
3.  **Builds Point Mappings**: For every image where corners are found, it stores the ideal 3D `obj_points` and the detected 2D `img_points`.
4.  **Calculates Parameters**: After processing all images, it passes these point mappings to `cv2.calibrateCamera()`. This powerful function uses all the data to mathematically solve for the camera's intrinsic parameters:
    * **`mtx` (Camera Matrix)**: Contains the focal lengths (fx, fy) and the optical center (cx, cy) of the camera.
    * **`dist` (Distortion Coefficients)**: Contains the values (k1, k2, p1, p2, k3) that mathematically describe the lens distortion.

### 4.3 FOV and Data Handling

* **`_calculate_fov`**: This new method takes the focal lengths (`fx`, `fy`) from the camera matrix and the image dimensions to calculate the camera's horizontal and vertical Field of View in degrees. This provides immediate, practical information about the camera's properties.
* **`save_parameters` / `load_parameters`**: These methods handle data persistence. The new version intelligently saves to either NumPy's binary `.npz` format or a human-readable `.json` format based on the filename, and can load from either.

### 4.4 Interactive Undistortion (`undistort_interactive`)

This mode showcases the practical application of calibration.
* It uses `cv2.getOptimalNewCameraMatrix()` with a trackbar-controlled `alpha` value. The `alpha` parameter controls how much of the original image is retained after undistortion.
    * `alpha=0`: The undistorted image is zoomed and cropped so that only valid pixels are visible (no black borders).
    * `alpha=1`: All original pixels are retained, which often results in black, curved borders where the image was warped.
* This interactivity allows a user to find the perfect balance for their specific needs.

### 4.5 Reprojection Error Visualization (`live_reprojection_error`)

This is a key method for assessing calibration quality.
1.  It first detects the chessboard corners in the live feed (`refined_corners`).
2.  It then uses `cv2.solvePnP()` to find the real-time 3D position and orientation of the board relative to the camera.
3.  Finally, it uses `cv2.projectPoints()` to take the *ideal* 3D object points (`self.objp`) and, using the calculated camera parameters and board position, re-projects them back into the 2D image plane.
4.  By drawing both the detected (green) and re-projected (red) points, it provides a direct visual measurement of the calibration error. **The smaller the distance between the green and red markers, the better the calibration.**

---

## Part 5: 🚀 Usage and Demonstration

This is a command-line tool. While we can't run the live camera modes directly in Colab (as it doesn't have access to your local webcam), we can explain exactly how you would run it on your own machine after downloading the script.

You would open a terminal or command prompt, navigate to the directory where you saved `camera_calibrator.py`, and run one of the following commands.

### 1. Live Calibration Mode
This is used to capture images and generate a new calibration file.

**Command:**
```bash
python camera_calibrator.py --mode live --rows 6 --cols 9 --size 25 --output my_camera.json

* A camera window will appear.

* Hold up a 9x6 chessboard and press `s` to save images from different angles.

* Press `q` to finish. The script will then calculate and save the parameters to `my_camera.json`.

### 2. Interactive Undistortion Mode
This is used to see the real-time effect of your calibration.

```bash
python camera_calibrator.py --mode undistort_interactive --output my_camera.json

* A camera window will appear showing the corrected video feed.

* Use the "Scaling (alpha)" slider at the top to adjust the cropping of the undistorted image.

### 3. Reprojection Error Mode
This is used to visually check the accuracy of your calibration.

```bash
python camera_calibrator.py --mode reprojection --rows 6 --cols 9 --output my_camera.json

* A camera window will appear.

* Hold up your chessboard. The script will draw green circles on the detected corners and red crosses on the ideal, re-projected corners.

* The closer the red crosses are to the center of the green circles, the more accurate your calibration is.

* Example Output:
```bash
Starting live calibration capture...
Press 's' to save a frame when a chessboard is detected.
Press 'q' to finish capturing and start calibration.

Saved: calib_images\image_0.png
Saved: calib_images\image_1.png
Saved: calib_images\image_2.png
Saved: calib_images\image_3.png
Saved: calib_images\image_4.png
Saved: calib_images\image_5.png
Saved: calib_images\image_6.png
Saved: calib_images\image_7.png
Saved: calib_images\image_8.png
Saved: calib_images\image_9.png
Saved: calib_images\image_10.png
Saved: calib_images\image_11.png
Saved: calib_images\image_12.png
Saved: calib_images\image_13.png
Saved: calib_images\image_14.png
Saved: calib_images\image_15.png

Captured 16 images. Now calibrating from directory 'calib_images'...
Found 26 images. Processing...

Performing camera calibration...
Calibration successful!

Camera Matrix:
 [[491.57192756   0.         256.8648797 ]
 [  0.         493.61906269 301.85631927]
 [  0.           0.           1.        ]]

Distortion Coefficients:
 [[-0.13249625  1.45789062  0.04407384 -0.00948739 -2.60878122]]

--- Field of View (FOV) ---
Horizontal FOV: 66.13 degrees
Vertical FOV:   51.86 degrees
Diagonal FOV:   78.27 degrees
---------------------------


Saving calibration data to 'calibration_data.npz'...
Data saved successfully.


---
## Part 6: ✅ Conclusion

We have successfully built and analyzed an advanced, professional-grade camera calibration tool. It is structured, reusable, and provides powerful interactive features for both calibration and analysis.

**Key Takeaways:**
* The project demonstrates an end-to-end computer vision workflow.
* It moves beyond basic scripting by using a class-based structure and command-line arguments.
* It incorporates advanced features like FOV calculation, interactive parameter tuning, and visual accuracy assessment with reprojection error, showcasing a deep understanding of the topic.

This tool serves as a robust foundation for any future computer vision projects that require a calibrated camera, such as augmented reality, 3D reconstruction, or robotic vision.

---

**Thank you for following along with this project!**
